In [1]:
import os
import json
import collections
import re
import jieba
import math
import torch
from torch import nn
import random
import pandas as pd 
import numpy as np
import torch.nn as nn
import torch.utils.data as Data
from collections import OrderedDict
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
file_path = 'truncated_setence.json'  
with open(file_path, 'r', encoding='utf-8') as file:
    sentence_data = json.load(file)

original_sentence, truncated_sentence, supplement_sentence = [],[],[]
for item in sentence_data:
    original_sentence.append(item["original_sentence"])
    truncated_sentence.append(item["truncated_sentence"])
    supplement_sentence.append(item["supplement_sentence"])

seq_date = [[a, b] for a, b in zip(truncated_sentence, supplement_sentence)]

# seq_date = [[sentence.strip() for sentence in sublist] for sublist in seq_date]
# print(seq_date)

In [3]:
jieba.load_userdict("userdict.txt")
def seg_text(texts):
    seg_setence = []
    for text in texts:
        seg_setence.append(["" + ' '.join(list(jieba.cut(i, cut_all=False))) for i in text])
    return seg_setence
segmented_setence =seg_text(seq_date)
# segmented_setence

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wangzhaohui\AppData\Local\Temp\jieba.cache
Loading model cost 0.661 seconds.
Prefix dict has been built successfully.


In [4]:
def tokenize(lines, token='word'):

    if token == 'word':
        return [i.split() for line in lines for i in line]

    elif token == 'char':
        return [list(i) for line in lines for i in line]

    else:
        print('错位：未知令牌类型：' + token)

tokens = tokenize(segmented_setence)

# print(tokens) 

In [5]:
class Vocab:

    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):

        if tokens is None:
            tokens = []

        if reserved_tokens is None:
            reserved_tokens = []

        counter = count_corpus(tokens)

        self.token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)

        uniq_tokens = reserved_tokens

        uniq_tokens += [token for token, freq in self.token_freqs
                        if freq >= min_freq and token not in uniq_tokens]

        self.idx_to_token, self.token_to_idx = [], dict()

        for token in uniq_tokens:

            self.idx_to_token.append(token)
            
            self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):

        return len(self.idx_to_token)

    def __getitem__(self, tokens):

        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):

        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

def count_corpus(tokens):

    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]

    return collections.Counter(tokens)

In [6]:
letter = [char for word in tokens for char in word]
letter.append('<pad>')
letter.append('<bos>')
letter.append('<eos>')
vocab = Vocab(letter,reserved_tokens=[' '])
letter2idx = vocab.token_to_idx
# Seq2Seq参数
n_step = max(len(token) for token in tokens)
n_hidden = 256
n_class = len(letter2idx) # n分类问题
batch_size = 3
# print(letter,letter2idx,n_step,n_class)

In [7]:
def make_data(tokens):
    enc_input_all, dec_input_all, dec_output_all = [], [], []

    for idx, token in enumerate(tokens):
        padded_token = token + ['<pad>'] * (n_step - len(token))
        
        if idx % 2 == 0:  # 奇数次循环
            enc_input = [letter2idx[n] for n in (padded_token + ['<eos>'])]
            enc_input_all.append(np.eye(n_class)[enc_input])
        else:  # 偶数次循环
            dec_input = [letter2idx[n] for n in (['<bos>'] + padded_token)]
            dec_output = [letter2idx[n] for n in (padded_token + ['<eos>'])]
            dec_input_all.append(np.eye(n_class)[dec_input])
            dec_output_all.append(dec_output)

    # make tensor
    return torch.Tensor(enc_input_all), torch.Tensor(dec_input_all), torch.LongTensor(dec_output_all)

enc_input_all, dec_input_all, dec_output_all = make_data(tokens)
'''
enc_input_all: [len(sample), n_step+1 (because of '<eos>'), n_class]
dec_input_all: [len(sample), n_step+1 (because of '<bos>'), n_class]
dec_output_all: [len(sample), n_step+1 (because of '<eos>')]
'''
# 打印示例张量形状
print("enc_input_all shape:", enc_input_all.shape)
print("dec_input_all shape:", dec_input_all.shape)
print("dec_output_all shape:", dec_output_all.shape)

enc_input_all shape: torch.Size([300, 55, 2569])
dec_input_all shape: torch.Size([300, 55, 2569])
dec_output_all shape: torch.Size([300, 55])


C:\Users\wangzhaohui\AppData\Local\Temp\ipykernel_171052\1606852058.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  return torch.Tensor(enc_input_all), torch.Tensor(dec_input_all), torch.LongTensor(dec_output_all)


In [8]:
class TranslateDataSet(Data.Dataset):
    def __init__(self, enc_input_all, dec_input_all, dec_output_all):
        self.enc_input_all = enc_input_all
        self.dec_input_all = dec_input_all
        self.dec_output_all = dec_output_all

    def __len__(self): # 返回数据集长度
        return len(self.enc_input_all)

    def __getitem__(self, idx):
        return self.enc_input_all[idx], self.dec_input_all[idx], self.dec_output_all[idx]

loader = Data.DataLoader(TranslateDataSet(enc_input_all, dec_input_all, dec_output_all), batch_size, True)

In [9]:
# Seq2Seq模型
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()
        self.encoder = nn.GRU(input_size=n_class, hidden_size=n_hidden, dropout=0.5) # encoder
        self.decoder = nn.GRU(input_size=n_class, hidden_size=n_hidden, dropout=0.5) # decoder
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, enc_input, enc_hidden, dec_input):
        # enc_input(=input_batch): [batch_size, n_step+1, n_class]
        # dec_inpu(=output_batch): [batch_size, n_step+1, n_class]
        enc_input = enc_input.transpose(0, 1) # enc_input: [n_step+1, batch_size, n_class]
        dec_input = dec_input.transpose(0, 1) # dec_input: [n_step+1, batch_size, n_class]

        # h_t : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        _, h_t = self.encoder(enc_input, enc_hidden)
        # outputs : [n_step+1, batch_size, num_directions(=1) * n_hidden(=128)]
        outputs, _ = self.decoder(dec_input, h_t)

        model = self.fc(outputs) # model : [n_step+1, batch_size, n_class]
        return model

model = Seq2Seq().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

C:\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [10]:
best_train_loss = float('inf')  # 初始化最佳训练损失为正无穷大

for epoch in tqdm(range(1000)):
    for enc_input_batch, dec_input_batch, dec_output_batch in loader:
        # 创建初始隐藏状态，形状为 [num_layers * num_directions, batch_size, n_hidden]
        h_0 = torch.zeros(1, batch_size, n_hidden).to(device)

        (enc_input_batch, dec_intput_batch, dec_output_batch) = (enc_input_batch.to(device), dec_input_batch.to(device), dec_output_batch.to(device))
        # enc_input_batch : [batch_size, n_step+1, n_class]
        # dec_intput_batch : [batch_size, n_step+1, n_class]
        # dec_output_batch : [batch_size, n_step+1], 不是独热编码

        pred = model(enc_input_batch, h_0, dec_intput_batch)
        # pred : [n_step+1, batch_size, n_class]
        pred = pred.transpose(0, 1) # [batch_size, n_step+1, n_class]

        # 计算损失
        loss = 0
        for i in range(len(dec_output_batch)):
          # pred[i] : [n_step+1, n_class]
          # dec_output_batch[i] : [n_step+1]
            loss += criterion(pred[i], dec_output_batch[i])
            
        if loss < best_train_loss:
            best_train_loss = loss.item()  # 更新最佳训练损失
            torch.save(model.state_dict(), 'best_model.pth')  # 保存模型参数到文件
            
        if (epoch + 1) % 100 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'loss:', '{:.10f}'.format(loss))

        # 清零梯度、反向传播、更新参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

 10%|▉         | 99/1000 [28:24<4:39:16, 18.60s/it]

Epoch: 0100 loss: 0.0923801437
Epoch: 0100 loss: 0.0910558254
Epoch: 0100 loss: 0.0906631425
Epoch: 0100 loss: 0.0819570869
Epoch: 0100 loss: 0.0821958184
Epoch: 0100 loss: 0.0722135603
Epoch: 0100 loss: 0.0743302256
Epoch: 0100 loss: 0.0811608806
Epoch: 0100 loss: 0.0722501576
Epoch: 0100 loss: 0.0890818164
Epoch: 0100 loss: 0.0665072650
Epoch: 0100 loss: 0.0920307785
Epoch: 0100 loss: 0.0910010636
Epoch: 0100 loss: 0.0656246319
Epoch: 0100 loss: 0.0833595544
Epoch: 0100 loss: 0.0794587061
Epoch: 0100 loss: 0.0991960466
Epoch: 0100 loss: 0.0855457708
Epoch: 0100 loss: 0.0695541799
Epoch: 0100 loss: 0.0640334934
Epoch: 0100 loss: 0.0611523613
Epoch: 0100 loss: 0.1502667367
Epoch: 0100 loss: 0.0657773018
Epoch: 0100 loss: 0.0580303371
Epoch: 0100 loss: 0.1246130913
Epoch: 0100 loss: 0.0670419186
Epoch: 0100 loss: 0.0554975569
Epoch: 0100 loss: 0.0989067033
Epoch: 0100 loss: 0.0885257572
Epoch: 0100 loss: 0.1026119441
Epoch: 0100 loss: 0.0565382093
Epoch: 0100 loss: 0.1358364522
Epoch: 0

 20%|█▉        | 199/1000 [58:54<4:20:02, 19.48s/it]

Epoch: 0200 loss: 0.0272315741
Epoch: 0200 loss: 0.0504177399
Epoch: 0200 loss: 0.0379528739
Epoch: 0200 loss: 0.0973199233
Epoch: 0200 loss: 0.0240494162
Epoch: 0200 loss: 0.0221504401
Epoch: 0200 loss: 0.0608200841
Epoch: 0200 loss: 0.0313315019
Epoch: 0200 loss: 0.0178172775
Epoch: 0200 loss: 0.0165920276
Epoch: 0200 loss: 0.0526096523
Epoch: 0200 loss: 0.0310219005
Epoch: 0200 loss: 0.0215399265
Epoch: 0200 loss: 0.0130192367
Epoch: 0200 loss: 0.0266260579
Epoch: 0200 loss: 0.0239056572
Epoch: 0200 loss: 0.0972378254
Epoch: 0200 loss: 0.0308038909
Epoch: 0200 loss: 0.0607747845
Epoch: 0200 loss: 0.0589593202
Epoch: 0200 loss: 0.0708752796
Epoch: 0200 loss: 0.0584984533
Epoch: 0200 loss: 0.0640100613
Epoch: 0200 loss: 0.0550998971
Epoch: 0200 loss: 0.0261010081
Epoch: 0200 loss: 0.0607839301
Epoch: 0200 loss: 0.0494235791
Epoch: 0200 loss: 0.0186231993
Epoch: 0200 loss: 0.0343991891
Epoch: 0200 loss: 0.1169991046
Epoch: 0200 loss: 0.0323625579
Epoch: 0200 loss: 0.1524031460
Epoch: 0

 30%|██▉       | 299/1000 [1:30:05<3:23:13, 17.39s/it]

...

 90%|████████▉ | 899/1000 [4:43:05<30:05, 17.87s/it]  

Epoch: 0900 loss: 0.0024026954
Epoch: 0900 loss: 0.0021925154
Epoch: 0900 loss: 0.0192634054
Epoch: 0900 loss: 0.0112755820
Epoch: 0900 loss: 0.0356159583
Epoch: 0900 loss: 0.0288033336
Epoch: 0900 loss: 0.0054152030
Epoch: 0900 loss: 0.0051681469
Epoch: 0900 loss: 0.0359861851
Epoch: 0900 loss: 0.0067925518
Epoch: 0900 loss: 0.0129532665
Epoch: 0900 loss: 0.0241356324
Epoch: 0900 loss: 0.0287680104
Epoch: 0900 loss: 0.0054557347
Epoch: 0900 loss: 0.0152276056
Epoch: 0900 loss: 0.2205704749
Epoch: 0900 loss: 0.0040136757
Epoch: 0900 loss: 0.0179188289
Epoch: 0900 loss: 0.0167670287
Epoch: 0900 loss: 0.0203663427
Epoch: 0900 loss: 0.0130998772
Epoch: 0900 loss: 0.0269456133
Epoch: 0900 loss: 0.0053872867
Epoch: 0900 loss: 0.0080391727
Epoch: 0900 loss: 0.0027709170
Epoch: 0900 loss: 0.0044427966
Epoch: 0900 loss: 0.0721106157
Epoch: 0900 loss: 0.0036094978
Epoch: 0900 loss: 0.0108781885
Epoch: 0900 loss: 0.0122149698
Epoch: 0900 loss: 0.0087686619
Epoch: 0900 loss: 0.0034643749
Epoch: 0

100%|█████████▉| 999/1000 [5:16:38<00:19, 19.32s/it]

Epoch: 1000 loss: 0.1196050420
Epoch: 1000 loss: 0.0234113187
Epoch: 1000 loss: 0.0375663042
Epoch: 1000 loss: 0.0242067128
Epoch: 1000 loss: 0.0415447019
Epoch: 1000 loss: 0.0515568517
Epoch: 1000 loss: 0.0847185254
Epoch: 1000 loss: 0.0889247656
Epoch: 1000 loss: 0.0615168624
Epoch: 1000 loss: 0.0570661724
Epoch: 1000 loss: 0.0157573521
Epoch: 1000 loss: 0.0329881199
Epoch: 1000 loss: 0.0403163955
Epoch: 1000 loss: 0.0361820981
Epoch: 1000 loss: 0.0371708050
Epoch: 1000 loss: 0.0266416352
Epoch: 1000 loss: 0.0211823843
Epoch: 1000 loss: 0.0769900978
Epoch: 1000 loss: 0.0545339547
Epoch: 1000 loss: 0.0385418013
Epoch: 1000 loss: 0.0392250828
Epoch: 1000 loss: 0.0199826173
Epoch: 1000 loss: 0.0299857259
Epoch: 1000 loss: 0.0305710062
Epoch: 1000 loss: 0.0273272488
Epoch: 1000 loss: 0.0328950733
Epoch: 1000 loss: 0.0624484718
Epoch: 1000 loss: 0.0694651976
Epoch: 1000 loss: 0.0323398001
Epoch: 1000 loss: 0.0221208520
Epoch: 1000 loss: 0.0289973710
Epoch: 1000 loss: 0.0340316296
Epoch: 1

100%|██████████| 1000/1000 [5:16:56<00:00, 19.02s/it]


In [18]:
def supplement(word):

    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    
    input_tokens = list(jieba.cut(word, cut_all=False))

    enc_input_all,dec_input_all = [],[]
    input_indices = [letter2idx[token] for token in input_tokens]

    padded_input_indices = input_indices + [letter2idx['<pad>']] * (n_step - len(input_indices)) + [letter2idx['<eos>']]

    padded_input_indices = [letter2idx['<bos>']] + input_indices + [letter2idx['<pad>']] * (n_step - len(input_indices))

    enc_input_all.append(np.eye(n_class)[padded_input_indices])
    dec_input_all.append(np.eye(n_class)[padded_input_indices])
    torch.Tensor(enc_input_all)
    torch.Tensor(dec_input_all)
    enc_input,dec_input = torch.Tensor(enc_input_all),torch.Tensor(dec_input_all)
    
    enc_input, dec_input = enc_input.to(device), dec_input.to(device)
    hidden = torch.zeros(1, 1, n_hidden).to(device)
    
    output = model(enc_input, hidden, dec_input)

    predict = output.data.max(2, keepdim=True)[1]

    decoded = [vocab.to_tokens(idx) for idx in predict]

    if '<eos>' in decoded:
        translated = ''.join(decoded[:decoded.index('<eos>')])
    else:
        translated = ''.join(decoded)
        
    if not translated.endswith('。'):
        translated += '。'

    return translated.replace('<pad>', '')

In [19]:
for input_sentence in truncated_sentence:
    next_sentence = supplement(input_sentence)
    print("【输入句子】", input_sentence)    
    print("【补全句子】", input_sentence + next_sentence)
    print("-----------")

【输入句子】 为贯彻落实党中央、国务院决策部署
【补全句子】 为贯彻落实党中央、国务院决策部署，做好机构、开发国资委部署，。
-----------
【输入句子】 坚持在推动高质量发展中强化就业优先导向
【补全句子】 坚持在推动高质量发展中强化就业优先导向，加快建设科技发展—出现当地公共、按。
-----------
【输入句子】 （国家发展改革委、科技部、工业和信息化部、财政部、
【补全句子】 （国家发展改革委、科技部、工业和信息化部、财政部、税务总局国务院卫生等在、国家、工业和信息化部、民政部、商务部、。
-----------
【输入句子】 结合实施区域协调发展、乡村振兴等战略
【补全句子】 结合实施区域协调发展、乡村振兴等战略，适应太湖就业与的违约责任旅游业。就业实施就业。
-----------
【输入句子】 继续实施“三支一扶”计划、农村特岗教师计划、大学生志愿服务西部计划等基层服务项目
【补全句子】 继续实施“三支一扶”计划、农村特岗教师计划、大学生志愿服务西部计划等基层服务项目，合理更一件、，），加大发展、核心等人力资源就业服务、。，提供退税高质量。
-----------
【输入句子】 对到中西部地区、艰苦边远地区、老工业基地县以下基层单位就业的高校毕业生
【补全句子】 对到中西部地区、艰苦边远地区、老工业基地县以下基层单位就业的高校毕业生，、有和政务系统、、产权保护方式和、、积极、计划投资毕业生实行就业。
-----------
【输入句子】 （中央组织部、最高人民法院、最高人民检察院、教育部、民政部、
【补全句子】 （中央组织部、最高人民法院、最高人民检察院、教育部、民政部、财政部自然、教育部、水稻离校分担、直辖市、财政部、。
-----------
【输入句子】 落实大众创业、万众创新相关政策
【补全句子】 落实大众创业、万众创新相关政策，深化建设单位“、创业就业、。
-----------
【输入句子】 支持高校毕业生自主创业
【补全句子】 支持高校毕业生自主创业，按规定毕业生就业就业补贴，。
-----------
【输入句子】 支持高校毕业生发挥专业所长从事灵活就业
【补全句子】 支持高校毕业生发挥专业所长从事灵活就业，对毕业生离校后、服务、就业服务、。
-----------
【输入句子】 （国家发展改革委、教育部、科技部、财政部、人

【输入句子】 针对进出口总额同比下降，付凌晖分析，这主要受两方面因素影响
【补全句子】 针对进出口总额同比下降，付凌晖分析，这主要受两方面因素影响：一是总额同比下降8.3%各级表示，营造各级经济体疫情社会主义积极保持。。
-----------
【输入句子】 去年5至7月份，随着积压订单集中释放，进出口增速连续加快
【补全句子】 去年5至7月份，随着积压订单集中释放，进出口增速连续加快，7日至7月份，粮食价格由退税来到向出提供数据增长建设和推进人民。
-----------
【输入句子】 从需求来看，服务业吸纳劳动力明显增加
【补全句子】 从需求来看，服务业吸纳劳动力明显增加，住宿基本就业交通运输业、国务院就业就业。
-----------
【输入句子】 全国城镇调查失业率为5.3%
【补全句子】 全国城镇调查失业率为5.3%，比上征管消费税经济同，增值税。
-----------
【输入句子】  付凌晖表示，从整个经济运行情况看
【补全句子】  付凌晖表示，从整个经济运行情况看，经济运行表示，世界货币环节，，广义支持。
-----------
【输入句子】 1至7月份，原煤产量同比增长3.6%，发电量增长3.8%；
【补全句子】 1至7月份，原煤产量同比增长3.6%，发电量增长3.8%；777月份，粮食价格对制造业由24.1%，营造道路24.1%，（。
-----------
【输入句子】 “面对高温天气和部分地区严重洪涝灾害
【补全句子】 “面对高温天气和部分地区严重洪涝灾害，要地方各外商投资投诉原则上各推动对、。
-----------
【输入句子】 针对下阶段经济走势
【补全句子】 针对下阶段经济走势，付凌晖或各地的直达对，。
-----------
【输入句子】 7月24日召开的中共中央政治局会议针对“积极扩大国内需求”作出部署
【补全句子】 7月24日召开的中共中央政治局会议针对“积极扩大国内需求”作出部署，月份月粮食价格，国务院投资企业上按以营造国家经济不足，适用安排的。
-----------
【输入句子】 出台若干措施促进家居、汽车、电子产品等重点领域消费
【补全句子】 出台若干措施促进家居、汽车、电子产品等重点领域消费；聚焦信用风险调动经济科技开发、结伴发展按民生的等工作。
-----------
【输入句子】 付凌晖表示
【补全句子】 付凌晖表

【输入句子】 加强督促检查评估
【补全句子】 加强督促检查评估，进一步把和人力资源发展。
-----------
【输入句子】 资金实行国库单独拨付
【补全句子】 资金实行国库单独拨付，省级结合重点下达、、。
-----------
【输入句子】 省级财政部门应按照相关资金管理办法要求
【补全句子】 省级财政部门应按照相关资金管理办法要求，切实加强结合与或部门、全球和其。
-----------
【输入句子】 该项补助纳入直达资金范围
【补全句子】 该项补助纳入直达资金范围，标识各地清单资金”，，。
-----------
【输入句子】 请在收到本通知后7日内
【补全句子】 请在收到本通知后7日内，研究走发展公告和到日、、。
-----------
【输入句子】 在下达直达资金时
【补全句子】 在下达直达资金时，应信用风险资金管理，。
-----------
【输入句子】 据央视新闻联播报道
【补全句子】 据央视新闻联播报道，11就业就业机构。
-----------
【输入句子】 北京国家会计学院李旭红教授告诉第一财经
【补全句子】 北京国家会计学院李旭红教授告诉第一财经，此次对民营企业进出口二十大—此次,。
-----------
【输入句子】 今年，在我国经济发展正面临需求收缩、供给冲击、预期转弱三重压力下，
【补全句子】 今年，在我国经济发展正面临需求收缩、供给冲击、预期转弱三重压力下，通过中国粮食价格征管经济恢复—能量压力自主影响指定总体包括道路，、、和一体化部署、。
-----------
【输入句子】 此次会议明确，免征符合条件的科技企业孵化器
【补全句子】 此次会议明确，免征符合条件的科技企业孵化器、大学再度“充分体现医疗器械的高度重视创新和创造。
-----------
【输入句子】 此次会议决定，继续放宽初创科技型企业认定标准，
【补全句子】 此次会议决定，继续放宽初创科技型企业认定标准，凡凡再度，、扎实初创科技企业营造创造信号提供数据经济企业。
-----------
【输入句子】 为了支持创业投资发展
【补全句子】 为了支持创业投资发展，国家计划的、。
-----------
【输入句子】 这项政策对投资发生时间截止日期原本为去年底
【补全句子】 这项政策对投资发生时间截止日期原本为去年底，此次可商品中国和建可可发展核心受理支持。
-----

【输入句子】 （教育部、民政部、财政部、人力资源社会保障部、人民银行
【补全句子】 （教育部、民政部、财政部、人力资源社会保障部、人民银行、共青团中央、全国工商联、保护、社会市场、就业、、。
-----------
【输入句子】 建立高校毕业生就业岗位归集机制
【补全句子】 建立高校毕业生就业岗位归集机制，广泛毕业生就业就业需求，，国务院。
-----------
【输入句子】 构建权威公信的高校毕业生就业服务平台
【补全句子】 构建权威公信的高校毕业生就业服务平台，密集做好清理二十大毕业生网上岗位办理。
-----------
【输入句子】 （教育部、工业和信息化部、人力资源社会保障部、国务院国资委
【补全句子】 （教育部、工业和信息化部、人力资源社会保障部、国务院国资委共青团中央共青团中央、全国工商联和离校部、民政部市场、市场决策、，。
-----------
【输入句子】  健全高校学生生涯规划与就业指导体系
【补全句子】  健全高校学生生涯规划与就业指导体系，开展教育和职业编码失业，引导。
-----------
【输入句子】 注重理论与实践相结合
【补全句子】 注重理论与实践相结合，、、地方等经济。
-----------
【输入句子】 深入实施离校未就业高校毕业生就业创业促进计划
【补全句子】 深入实施离校未就业高校毕业生就业创业促进计划，、有教育就业创业毕业生及岗位担保经济等。
-----------
【输入句子】 运用线上失业登记、求职登记小程序、基层摸排等各类渠道
【补全句子】 运用线上失业登记、求职登记小程序、基层摸排等各类渠道，与和青年，失业就业等、、政府就业基础设施青年救灾、，。
-----------
【输入句子】 开展平等就业相关法律法规和政策宣传
【补全句子】 开展平等就业相关法律法规和政策宣传，坚决就业单位、、政策服务服务。
-----------
【输入句子】 督促用人单位与高校毕业生签订劳动（聘用）合同或就业协议书
【补全句子】 督促用人单位与高校毕业生签订劳动（聘用）合同或就业协议书，明确与高校毕业生离校后年龄不监管等等直属机构社保创造。
-----------
【输入句子】 取消高校毕业生离校前公共就业人才服务机构在就业协议书上签章环节
【补全句子】 取消高校毕业生离校前公共就业人才服务机构在就业协议书上签章环节，高校毕业生离

In [20]:
input_sentence = '教育部门要健全高校毕业生网上签约系统'
next_sentence = supplement(input_sentence)
print("【输入句子】", input_sentence)    
print("【补全句子】", input_sentence + next_sentence)

【输入句子】 教育部门要健全高校毕业生网上签约系统
【补全句子】 教育部门要健全高校毕业生网上签约系统，方便及时教育毕业生等签约，就业。


In [21]:
input_sentence = '各地区各部门要深入贯彻落实习近平总书记重要指示批示精神'
next_sentence = supplement(input_sentence)
print("【输入句子】", input_sentence)    
print("【补全句子】", input_sentence + next_sentence)

【输入句子】 各地区各部门要深入贯彻落实习近平总书记重要指示批示精神
【补全句子】 各地区各部门要深入贯彻落实习近平总书记重要指示批示精神，坚持推动部门的进一步加大中央经济来到地区核心的作为。
